In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
5+6

11

In [3]:
import nltk
nltk.__version__

'3.8.1'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB


In [5]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("training.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text
73820,0,1694637805,Mon May 04 03:30:54 PDT 2009,NO_QUERY,clrzqstn,"I think, we'll be out of town on the 16th. Th..."
1441013,4,2061758921,Sat Jun 06 21:33:06 PDT 2009,NO_QUERY,MeganCrossett,HOME!! Thanks @saintsammy and @eliza_electric ...
259100,0,1985345167,Sun May 31 16:55:36 PDT 2009,NO_QUERY,xolove,I seriously miss @teamjacobsucks so much I mig...
733352,0,2264292859,Sun Jun 21 03:24:04 PDT 2009,NO_QUERY,cominu,back from Milan...dead tired
1451370,4,2063034018,Sun Jun 07 01:08:08 PDT 2009,NO_QUERY,licksharder,@NaiveLondonGirl good for you girl! recommend ...
720149,0,2260806816,Sat Jun 20 19:48:12 PDT 2009,NO_QUERY,MissElyon,@Mcpattz J.D salinger? awww that suck
1543919,4,2181546274,Mon Jun 15 11:58:34 PDT 2009,NO_QUERY,karinonoda,aaaand for @jonas brothers for the best album ...
1428145,4,2059514508,Sat Jun 06 17:05:14 PDT 2009,NO_QUERY,marilynstjulian,"Dads in Africa for 2 weeks = boosin', sexin', ..."
353906,0,2033496016,Thu Jun 04 12:47:14 PDT 2009,NO_QUERY,shallowend,"Very sneezy today, and my throat just started ..."
743728,0,2266930834,Sun Jun 21 09:20:22 PDT 2009,NO_QUERY,roseclear,SUCH a good sleep! I don't want to leave Brian.


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [7]:
df.shape

(1600000, 6)

In [8]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [10]:
df.sample(10)

,target,text
1234065,4,"@nicosiaoceania Oh, i'ts great!!! I'm in 4th ..."
9852,0,Yeah had to call into work today. The Dr. is g...
1370745,4,"Okay so im alil drunk right now, been drinking..."
384862,0,Good morning all! Plans for today have been sc...
20730,0,Has loads of coursework to do
1215659,4,@TimothyH2O watch the andy samberg opening act...
653557,0,wish it wasn't so crappy outside iwanna go ...
1019085,4,@barb72 Lol just to cap off a gr8 week how u ...
756659,0,Grillin some turkey sausage... what a day
909977,4,just got home from Tillie's dance recital. lol...


In [11]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [12]:
df.sample(10)

,sentiment,tweet
295777,0,Aw whats wrong ~LoveMySexyJellyBean~
677126,0,"@NYCP Let's be clear..even if he didn't win, h..."
993150,4,my playlist is full of Taylor Swift's songs! +...
1539119,4,"Ok,....I think I've calmed down a little. My ..."
581488,0,@OfficialNichole damn i really wish i could go...
382434,0,Home now. Wanting to be with B.
1467911,4,@micaella_ella i had such a hard time choosing...
679076,0,It's like 80 degrees in my room sleep seems l...
78018,0,Elena lost the balloon I gave her she couldn'...
1569884,4,@bankervision you're lucky you have the tube f...


In [13]:
df = df.dropna()
df.sample(10)

,sentiment,tweet
1020046,4,@Hargarmoopy DON'T say ONE wrong word about Jo...
530949,0,@sandygrason Don't know if I'll be available t...
444284,0,Wanting to see Phish
290645,0,@rpgdude my spanish aren't that good I know M...
215080,0,@kayden_kross Watch your back! That rabbit sou...
1354090,4,"http://twitpic.com/6osxf - Me,Sammeh,and Miss ..."
242909,0,i'm laying around because scott took the fishi...
1141980,4,I would like a plum. They are more dark blue t...
386499,0,Ugh....snow in June...it's just not right.
1131139,4,jus got out of the shower work was busy...my f...


In [14]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None
The Shape Of The Dataset Is : (1600000, 2)


In [15]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [16]:
df.sample(5)

,sentiment,tweet
1342866,positive,just chillin until softball tonite!!!!!
1200638,positive,definitely in the top three! vi@paulisded: &q...
530751,negative,playin hooky and chillin with Lauren before sh...
1064530,positive,"@Herbguy Curious, what facility do you use now..."
1098244,positive,@mthe71 enjoy fika! we have the sun here too-w...


In [17]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [18]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet
1473331  positive    JohnnyBTruant she thinks her mothers hilarious 
646657   negative  I just dreamt I was at a Growing Pains reunion...
192293   negative  Argh Buried under the contracts in iTunes Conn...
52291    negative                          i want to get snakebites 
1381863  positive    devilworks                                 
474153   negative  Looking forward to a day of family assessments...
310682   negative  hangyourcross I should just reply to you on AI...
249192   negative  Played a soccergame today i hurt my leg  Now i...
693291   negative  Kids warn your grandparents about the CDN Car ...
325155   negative  screw history revisionim gonna mourn the loss ...
The information of the dataset is :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentim

In [19]:
df['tweet'] = df['tweet'].str.lower()

In [20]:
df.sample(10)

,sentiment,tweet
591323,negative,moorephoto after i just left
908283,positive,ahhh bank holiday slept late now going to dinn...
1550917,positive,cmmpr i agree for business folk it has to end ...
607168,negative,puppies didnt know some of their names
238846,negative,marleythelemur im actually not that sore my fe...
1100675,positive,mr twitter universe competition ends in days ...
1258102,positive,tbh cheap chocolate tastes nicer exept wonker
1387335,positive,farnush it is the one that we went adabor
1597754,positive,divokc our office is soooo busy we wouldnt hav...
1467393,positive,new blog post up and flickr upload


In [21]:
# stop_words = set(stopwords.words('english'))
# df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [22]:
# Extract features using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = vectorizer.fit_transform(df['tweet'])
y = df['target']

: 

: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)